In [1]:
from tpu_utility_1 import *
# setup_kaggle()

import os

try: import jpeg4py as jpeg
except Exception:
    os.system("apt-get install libturbojpeg")
    os.system("pip install jpeg4py")
from IPython.display import clear_output
clear_output()

In [2]:
import numpy as np 
import os
import pandas as pd 
import matplotlib.pyplot as plt

import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.utils as xu
import torch_xla.utils.cached_dataset as xcd

import torch
import torch.nn as nn
import torch.utils.data as D
from torch.optim import lr_scheduler
from torchvision import models

import albumentations
import jpeg4py as jpeg  # decoration import. 
from PIL import Image
from tqdm.notebook import tqdm
from pathlib import Path
from sklearn.model_selection import StratifiedKFold

# from sklearn.metrics import f1_score

In [3]:
diseases = ['healthy', 'rust', 'scab', 'frog_eye_leaf_spot', 'powdery_mildew', 'complex']
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

labels = df.labels.str.split()
occ = np.zeros((len(labels),len(diseases)))
count = 0
for i in diseases:
    occ[:,count] = [i in list for list in labels]
    count+=1

dis_df = pd.DataFrame(occ.astype(np.float32), columns=diseases)

torch_df = pd.concat([df["image"], dis_df], axis=1)
torch_df.head()

,image,healthy,rust,scab,frog_eye_leaf_spot,powdery_mildew,complex
0,800113bb65efe69e.jpg,1.0,0.0,0.0,0.0,0.0,0.0
1,8002cb321f8bfcdf.jpg,0.0,0.0,1.0,1.0,0.0,1.0
2,80070f7fb5e2ccaa.jpg,0.0,0.0,1.0,0.0,0.0,0.0
3,80077517781fb94f.jpg,0.0,0.0,1.0,0.0,0.0,0.0
4,800cbf0ff87721f8.jpg,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(torch_df)
tdf_np = df.to_numpy()
X = tdf_np[:, 0]
y = tdf_np[:, 1]
for train_index, test_index in skf.split(X, y): pass
torch_df.to_numpy()[1][1:].astype(np.float32)

array([0., 0., 1., 1., 0., 1.], dtype=float32)

# Model Definition

In [5]:
num_classes = len(diseases)

model = models.resnet50(pretrained=True)
num_features = model.fc.in_features

# freeze models
for param in model.parameters():
    param.requires_grad_(False)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [6]:
def create_head(num_features, num_classes, dropout=0.1, act_func=nn.ReLU):
    features_lst = [num_features, num_features // 2, num_features // 4]
    layers = []
    
    for in_f, out_f in zip(features_lst[:-1], features_lst[1:]):
        layers.append(nn.Linear(in_f, out_f))
        layers.append(act_func())
        layers.append(nn.BatchNorm1d(out_f))
        if dropout != 0: layers.append(nn.Dropout(dropout))
            
    layers.append(nn.Linear(features_lst[-1], num_classes))
    return nn.Sequential(*layers)


top_head = create_head(num_features, num_classes)
model.fc = top_head

WRAPPED_MODEL = xmp.MpModelWrapper(model)

In [7]:
FLAGS = {
    "data_dir": Path("../input/plant-pathology-2021-fgvc8/train_images"),
    "bs": 32,
    "num_workers": os.cpu_count(),
    "lr": 0.001,
    "momentum": 0.9,  # if using SGD
    "num_epochs": 10,
    "num_cores": 8 if os.environ.get("TPU_NAME", None) else 1,
    "log_steps": 20,  # not used? 
    "metrics_debug": False,
    "seed": 1397,
    "save_path": "/kaggle/working/model.pth"
}

In [8]:
class AppleDataset(D.Dataset):
    def __init__(self, parent_path, df=None, shuffle=False, seed=None, transforms=None, 
                split=None):
        """
        parent_path: (pathlib.Path) Parent path of images. 
        df: (pandas.DataFrame) y-labels dataframe. 
        shuffle: (Boolean) Shuffle dataset? Default: False. 
        seed: (integer) seed. Default: None. 
        transforms: (Pytorch Transforms) Transformation to images, default: None. 
        split: (python list) Splits index for train test split. 
        """
        self.df_np = df.to_numpy()
        self.parent_path = parent_path
        self.seed = seed
        self.transforms = transforms
        
        if type(split) != type(None): self.df_np = self.df_np[split]
        
    def __len__(self):
        return len(self.df_np)
    
    def __getitem__(self, idx):
        item = self.df_np[idx]
        item_path = self.parent_path / item[0]  # 'image' column
        image = jpeg.JPEG(item_path).decode()
        
        if self.transforms is not None: image = self.transforms(**{"image": image})

        image = torch.from_numpy(image["image"]).permute(2, 0, 1)  # HWC -> CHW format
        target = torch.from_numpy(item[1:].astype(np.float32))
        
        return image, target

In [9]:
size = 224

train_transform = albumentations.Compose([
    albumentations.RandomResizedCrop(height=size, width=size, always_apply=True, scale=(0.75, 1.0)),
    albumentations.Flip(),
    albumentations.Rotate(limit=10, p=0.75),
    albumentations.RandomBrightnessContrast(p=0.75),
    albumentations.Normalize(always_apply=True, p=1.0)
])

# val_transform = albumentations.Compose([
#     albumentations.RandomResizedCrop(height=size, width=size, always_apply=True, scale=(0.9, 1.0)),
#     albumentations.Normalize(always_apply=True, p=1.0)
# ])

In [10]:
train_ds = AppleDataset(FLAGS["data_dir"], torch_df, shuffle=True, 
                        transforms=train_transform, split=train_index)
# val_ds = AppleDataset(FLAGS["data_dir"], torch_df, shuffle=False, split=test_index,
#                      transforms=val_transform)

# dls = dataloader(train_ds, val_ds, FLAGS)

In [11]:
criterion = nn.BCEWithLogitsLoss()
opt = torch.optim.Adam(model.parameters(), lr=FLAGS["lr"])

# Learning Rate Finder

In [12]:
# cache_train_loc = Path("../input/tpu-applepie/cached-train/cached-train")
# cache_val_loc = Path("../input/tpu-applepie/cached-val/cached-val")

# def _mp_fn(index):
#     global FLAGS
#     train_ds, val_ds = cached_dataset(cache_train_loc, cache_val_loc)
#     dls = dataloader(train_ds, val_ds, FLAGS)
#     lrfinder = LRFinder(model, opt, criterion, device=xm.xla_device())
#     lrfinder.lr_find(dls["train"])
    
# xmp.spawn(_mp_fn, args=(), start_method='fork', nprocs=1)

In [13]:
# lrfinder = LRFinder(model, opt, criterion, device=xm.xla_device())
# model = lrfinder.lr_find(dls["train"])

# One Cycle Policy

In [14]:
cache_train_loc = Path("../input/tpu-applepie/cached-train/cached-train")
cache_val_loc = Path("../input/tpu-applepie/cached-val/cached-val")
fit_one_cycle = OneCyclePolicy_TPU(model, opt, criterion, FLAGS, train_ds=train_ds,
                                  cache_train_loc=cache_train_loc, 
                                  cache_val_loc=cache_val_loc)

In [15]:
fit_one_cycle.train()

  0%|          | 0/116 [00:00<?, ?it/s]